In [1]:
import keras
# from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
# from livelossplot.keras import PlotLossesCallback
from sklearn.preprocessing import LabelEncoder


In [2]:
# import tensorflow as tf
# tf.compat.v1.get_default_graph()

In [2]:
le = LabelEncoder()

In [4]:
train = pd.read_excel("data/Train.xlsx") #to read the data from the file 

In [5]:
train["image"]=train["image"].astype(str) #covert the data to string 
train["class"]=train["class"].astype(str)

In [6]:
train #display training data

,image,class
0,0,6
1,1,9
2,2,X
3,3,V
4,4,3
...,...,...
1104,1992,N
1105,1993,N
1106,1994,6
1107,1995,T


In [8]:
train_image = [] # reading images from trian folder and converting them into list of arrays
for i in tqdm(range(train.shape[0])):
    img = image.load_img("data/train/"+train["image"][i]+".jpg",target_size= (10,15,1),grayscale=True) #defining size of target as all the images should be of same size                    
    img = image.img_to_array(img)
    img = img/225                 # normalising the image
    train_image.append(img)    #appending each image array into train_image 

100%|█████████████████████████████████████████████████████████████████████████████| 1109/1109 [00:09<00:00, 112.40it/s]


In [9]:
val = pd.read_excel("data/Val.xlsx") #loading validation or test data

In [10]:
val["image"]=val["image"].astype(str)
val["class"]=val["class"].astype(str)

In [11]:
val

,image,class
0,2001,N
1,2002,X
2,2003,K
3,2004,Z
4,2005,F
...,...,...
435,2795,3
436,2800,P
437,2801,6
438,2802,3


In [12]:
val_image = []
for j in tqdm(range(val.shape[0])):
    img1 = image.load_img("data/test/"+val["image"][j]+".jpg",target_size= (10,15,1),grayscale=True)
    img1 = image.img_to_array(img1)
    img1 = img1/225
    val_image.append(img1)

100%|███████████████████████████████████████████████████████████████████████████████| 440/440 [00:03<00:00, 126.86it/s]


In [13]:
x = np.array(train_image)  # Assigning train data "x" as a numpy array 
# y = train["class"].values   # getting the values of the train images from train dataframe)
d = le.fit_transform(train["class"])
# y = le.fit_transform(y)
y = to_categorical(d)   
y   # converting the values to categorial

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
x.shape,y.shape  # checking the shape of the input data for the CNN model

((1109, 10, 15, 1), (1109, 34))

In [15]:
X = np.array(val_image)   # Assigning val data "X" as a numpy array 
D = le.fit_transform(val["class"])   # getting the values of the validation images from val dataframe
Y = to_categorical(D)     # converting the values to categorial

In [16]:
X.shape,Y.shape  # checking the shape of the validation data for the CNN model

((440, 10, 15, 1), (440, 34))

In [17]:
model = Sequential()      # assigning a object instance to variable model
model.add(Conv2D(32,kernel_size=(2,2),activation = "relu",input_shape = (10,15,1))) # input layer
model.add(Conv2D(128,(3,3),activation = "relu")) # 1st layer with 128 convolution filters and the 3x3 kernal size and using relu activation function
model.add(Conv2D(64,(3,3),activation = "relu"))  # 2nd layer with 64 convolution filters and the 3x3 kernal size and using relu activation function
# model.add(Conv2D(128,(3,3),activation = "relu")) # 3rd layer with 128 convolution filters and the 3x3 kernal size and using relu activation function
# model.add(MaxPooling2D(pool_size = (3,3),strides=2))
model.add(Conv2D(64,(3,3),activation = "relu"))  # 2nd layer with 64 convolution filters and the 3x3 kernal size and using relu activation function
model.add(Conv2D(128,(3,3),activation = "relu"))# using max pooling with 2x2 kernal size
# model.add(MaxPooling2D(pool_size = (3,3),strides=2))
model.add(Dropout(0.35))                   # using Dropout function to decrease bias(drop neurons which dont have information)
model.add(Flatten())                       # after convolution layer making it as flatten (all the pixcels values are in one column while inputing into neural network)
model.add(Dense(256,activation ="relu"))   # 1st  dense layer with 256 nerouns and activation function as relu 
# model.add(Dense(256,activation ="relu"))   # 2nd  dense layer with 256 nerouns and activation function as relu 
# model.add(Dense(64,activation ="relu"))    # 3rd  dense layer with 256 nerouns and activation function as relu 
model.add(Dropout(0.45))                   # using Dropout function to decrease bias
# model.add(Dense(64,activation ="relu"))   # 4th  dense layer with 256 nerouns and activation function as relu 
model.add(Dense(34,activation ="softmax")) # output layer with 10 nerouns and activation function softmax

In [18]:
model.compile(loss = "categorical_crossentropy",optimizer = "adam",metrics = ["accuracy"])

In [19]:
model.fit(x,y,epochs=30,validation_data=(X,Y))

Epoch 1/30
35/35 [==============================] - 4s 45ms/step - loss: 2.7065 - accuracy: 0.2741 - val_loss: 1.0586 - val_accuracy: 0.7500
Epoch 2/30
35/35 [==============================] - 1s 33ms/step - loss: 0.9057 - accuracy: 0.7610 - val_loss: 0.3047 - val_accuracy: 0.9341
Epoch 3/30
35/35 [==============================] - 1s 34ms/step - loss: 0.4814 - accuracy: 0.8729 - val_loss: 0.2444 - val_accuracy: 0.9591
Epoch 4/30
35/35 [==============================] - 1s 35ms/step - loss: 0.3076 - accuracy: 0.9270 - val_loss: 0.1959 - val_accuracy: 0.9773
Epoch 5/30
35/35 [==============================] - 1s 33ms/step - loss: 0.2641 - accuracy: 0.9441 - val_loss: 0.1741 - val_accuracy: 0.9886
Epoch 6/30
35/35 [==============================] - 1s 33ms/step - loss: 0.1760 - accuracy: 0.9639 - val_loss: 0.1719 - val_accuracy: 0.9886
Epoch 7/30
35/35 [==============================] - 1s 33ms/step - loss: 0.1537 - accuracy: 0.9766 - val_loss: 0.1518 - val_accuracy: 0.9864
Epoch 8/30
35

In [21]:
# serialize weights to HDF5
model.save_weights("data/Model/model_weight_25_06_2021.h5") #save weights for prediction
print("Saved model to disk")
# serialize model to HDF5 which is format to easily store weights
model.save("data/Model/model2_10_15_25_06_2021.h5")

Saved model to disk


In [22]:
test = pd.read_excel("data/testing.xlsx")

In [23]:
test["image"] = test["image"].astype(str)
test["class"] = test["class"].astype(str)

In [24]:
test

,image,class
0,9632,Z
1,9633,3
2,9634,3
3,9635,5
4,9640,8
...,...,...
193,9991,R
194,9992,6
195,9993,U
196,9994,H


In [25]:
test_image = []
for p in tqdm(range(test.shape[0])):
    img2 = image.load_img('data/New folder/'+test['image'][p]+'.jpg', target_size=(10,15,1),grayscale=True)
    img2 = image.img_to_array(img2)
    img2 = img2/255
    test_image.append(img2)
test1 = np.array(test_image)

100%|███████████████████████████████████████████████████████████████████████████████| 198/198 [00:01<00:00, 126.60it/s]


In [26]:
test1.shape

(198, 10, 15, 1)

In [31]:
#Predicting the values of all images and store it in pre from test1
predict = model.predict(test1)
pre=np.argmax(predict,axis=1)
# pre = le.inverse_transform(classes)

7/7 [==============================] - 0s 8ms/step


In [32]:
pre

array([33,  3,  3,  5,  8, 28,  2,  7, 18, 20,  0,  0,  6, 28,  1,  7, 11,
        9, 28,  3,  3, 27,  0, 10,  4, 18, 20, 13, 23,  5, 28, 14,  1, 24,
       11, 13,  2,  5,  1, 15,  5, 20,  6,  2, 29, 14,  1,  8,  6,  9, 10,
       28, 20, 11,  4,  4,  2,  3,  9,  4, 17,  2,  0,  1,  8,  9,  9,  8,
        5, 27, 28,  4,  5,  7,  1, 20,  0, 25, 21,  3, 18,  0,  0, 21, 33,
        3, 18,  0, 23,  4,  7,  7, 20,  4, 31, 33,  2,  6, 14, 32,  3,  1,
        8, 30, 31,  3, 12,  1, 25,  4, 15, 33, 16,  2,  3, 10,  8, 15,  7,
       20, 27, 30,  4, 17, 17, 12, 24,  8,  8, 17,  7, 20, 10, 16,  2,  2,
       10,  1,  4, 31, 26,  2,  5,  1,  4,  4,  4, 29,  3,  5,  8, 25, 10,
       29,  6, 31,  7,  7,  0,  0,  0, 23, 22,  3, 24, 28, 16, 32, 26,  7,
       23, 23, 10, 24, 16,  7,  8, 10, 15, 14, 18, 12,  0,  0, 10,  2,  5,
        1,  9, 10, 33, 17, 24, 25,  6, 28, 17,  7], dtype=int64)

In [33]:
pre

array([33,  3,  3,  5,  8, 28,  2,  7, 18, 20,  0,  0,  6, 28,  1,  7, 11,
        9, 28,  3,  3, 27,  0, 10,  4, 18, 20, 13, 23,  5, 28, 14,  1, 24,
       11, 13,  2,  5,  1, 15,  5, 20,  6,  2, 29, 14,  1,  8,  6,  9, 10,
       28, 20, 11,  4,  4,  2,  3,  9,  4, 17,  2,  0,  1,  8,  9,  9,  8,
        5, 27, 28,  4,  5,  7,  1, 20,  0, 25, 21,  3, 18,  0,  0, 21, 33,
        3, 18,  0, 23,  4,  7,  7, 20,  4, 31, 33,  2,  6, 14, 32,  3,  1,
        8, 30, 31,  3, 12,  1, 25,  4, 15, 33, 16,  2,  3, 10,  8, 15,  7,
       20, 27, 30,  4, 17, 17, 12, 24,  8,  8, 17,  7, 20, 10, 16,  2,  2,
       10,  1,  4, 31, 26,  2,  5,  1,  4,  4,  4, 29,  3,  5,  8, 25, 10,
       29,  6, 31,  7,  7,  0,  0,  0, 23, 22,  3, 24, 28, 16, 32, 26,  7,
       23, 23, 10, 24, 16,  7,  8, 10, 15, 14, 18, 12,  0,  0, 10,  2,  5,
        1,  9, 10, 33, 17, 24, 25,  6, 28, 17,  7], dtype=int64)

In [34]:
from sklearn.metrics import accuracy_score,confusion_matrix,average_precision_score  # to check the accuracy 

In [35]:
test["pre"] = le.inverse_transform(pre) # we are decoding the images encoded in the beginning
test["class"] = test["class"].astype(str)

In [36]:
test

,image,class,pre
0,9632,Z,Z
1,9633,3,3
2,9634,3,3
3,9635,5,5
4,9640,8,8
...,...,...,...
193,9991,R,R
194,9992,6,6
195,9993,U,U
196,9994,H,H


In [37]:
actual = np.array(test["class"]) # assigning actual values 
p = np.array(test["pre"])        # assigning predicted values

In [38]:
p,actual

(array(['Z', '3', '3', '5', '8', 'U', '2', '7', 'J', 'L', '0', '0', '6',
        'U', '1', '7', 'B', '9', 'U', '3', '3', 'T', '0', 'A', '4', 'J',
        'L', 'D', 'P', '5', 'U', 'E', '1', 'Q', 'B', 'D', '2', '5', '1',
        'F', '5', 'L', '6', '2', 'V', 'E', '1', '8', '6', '9', 'A', 'U',
        'L', 'B', '4', '4', '2', '3', '9', '4', 'H', '2', '0', '1', '8',
        '9', '9', '8', '5', 'T', 'U', '4', '5', '7', '1', 'L', '0', 'R',
        'M', '3', 'J', '0', '0', 'M', 'Z', '3', 'J', '0', 'P', '4', '7',
        '7', 'L', '4', 'X', 'Z', '2', '6', 'E', 'Y', '3', '1', '8', 'W',
        'X', '3', 'C', '1', 'R', '4', 'F', 'Z', 'G', '2', '3', 'A', '8',
        'F', '7', 'L', 'T', 'W', '4', 'H', 'H', 'C', 'Q', '8', '8', 'H',
        '7', 'L', 'A', 'G', '2', '2', 'A', '1', '4', 'X', 'S', '2', '5',
        '1', '4', '4', '4', 'V', '3', '5', '8', 'R', 'A', 'V', '6', 'X',
        '7', '7', '0', '0', '0', 'P', 'N', '3', 'Q', 'U', 'G', 'Y', 'S',
        '7', 'P', 'P', 'A', 'Q', 'G', '7', '8', 'A'

In [39]:
results = confusion_matrix(actual, p)
results

array([[13,  0,  0, ...,  0,  0,  0],
       [ 0, 11,  0, ...,  0,  0,  0],
       [ 0,  0, 11, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  4,  0,  0],
       [ 0,  0,  0, ...,  0,  2,  0],
       [ 0,  0,  0, ...,  0,  0,  5]], dtype=int64)

In [40]:
acc = accuracy_score(actual,p)
print('Average precision-recall score: {0:0.2f}'.format(acc))

Average precision-recall score: 1.00
